In [1]:
from cuopt import routing
from cuopt import distance_engine
import cudf
from scipy.spatial import distance
import numpy as np
import requests
import polyline
import folium
import json

# Cost Matrix Calculation

The cost matrix represents the user defined cost of traversing from one state/location in the optimization problem to another. This matrix is what cuOpt uses to assess the quality of a given solution as it seeks to minimize the total cost.

The cost matrix is a square matrix of dimension equal to the number of locations in a given problem. In the example below we see an illustration of one such matrix.

<img src="./notebook_utils/images/cost_matrix.png" alt="cost_matrix.png not found]" width="750"/>

Additionally:
- The cost of going from a location to itself (e.g Cost(A,A)) is typically 0 
- Cost(A,B) need not be equal to Cost(B,A)

## Simple Metric

In some simple cases a cost matrix can be generated from a list of points according to a user defined metric (e.g. Euclidean, Manhattan, etc.)

In [2]:
points = cudf.DataFrame({"x_coord": [1, 1, 2, 3], "y_coord":[3, 1, 4, 1]})
points

,x_coord,y_coord
0,1,3
1,1,1
2,2,4
3,3,1


In [3]:
cost_matrix = distance.cdist(points.to_pandas().values, points.to_pandas().values, "euclidean")
print(f"Simple Metric Cost Matrix:\n\n{cost_matrix}")

Simple Metric Cost Matrix:

[[0.         2.         1.41421356 2.82842712]
 [2.         0.         3.16227766 2.        ]
 [1.41421356 3.16227766 0.         3.16227766]
 [2.82842712 2.         3.16227766 0.        ]]


## Weighted Waypoint Graph

In cases where a unique environment needs to be described such as in the case of factories or warehouses it can be useful to define a waypoint graph that defines the cost of travel between adjacent accessible points in the environment.

cuOpt has built in functionality to compute a cost matrix based on key target locations within a given waypoint graph. In the graph below we model 10 distinct waypoints.  The target locations are 0, 4, 5, and 6.

<img src="./notebook_utils/images/waypoint_graph.png" alt="waypoint_graph.png not found]" width="550"/>

#### Graph Description
A simple description of each node, it's outgoing edges and corresponding weights

In [4]:
graph = {
    0:{
        "edges":[2], 
        "weights":[2]},
    1:{
        "edges":[2, 4], 
        "weights":[2, 2]},
    2:{
        "edges":[0, 1, 3, 5], 
        "weights":[2, 2, 2, 2]},
    3:{
        "edges":[2, 6], 
        "weights":[2, 2]},
    4:{
        "edges":[1, 7], 
        "weights":[2, 1]},
    5:{
        "edges":[2, 8], 
        "weights":[2, 1]},
    6:{
        "edges":[3, 9], 
        "weights":[2, 1]},
    7:{
        "edges":[4, 8], 
        "weights":[1, 2]},
    8:{
        "edges":[5, 7, 9], 
        "weights":[1, 2, 2]},
    9:{
        "edges":[6, 8], 
        "weights":[1, 2]}
}

#### Convert to CSR
cuOpt requires that the graph be in compressed sparse row (CSR) format.  Here we define a simple function that converts our graph to CSR.

In [5]:
def convert_to_csr(graph):
    num_nodes = len(graph)
    
    offsets = []
    edges = []
    weights = []
    
    cur_offset = 0
    for node in range(num_nodes):
        offsets.append(cur_offset)
        cur_offset += len(graph[node]["edges"])
        
        edges = edges + graph[node]["edges"]
        weights = weights + graph[node]["weights"]
        
    offsets.append(cur_offset)
    
    return np.array(offsets), np.array(edges), np.array(weights)

In [6]:
offsets, edges, weights = convert_to_csr(graph)
print(f"offsets = {list(offsets)}")
print(f"edges =   {list(edges)}")
print(f"weights = {list(weights)}")

offsets = [0, 1, 3, 7, 9, 11, 13, 15, 17, 20, 22]
edges =   [2, 2, 4, 0, 1, 3, 5, 2, 6, 1, 7, 2, 8, 3, 9, 4, 8, 5, 7, 9, 6, 8]
weights = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2]


#### Define desired target locations and calculate the cost matrix 

In [7]:
target_locations = np.array([0, 4, 5, 6])

In [8]:
waypoint_graph = distance_engine.WaypointMatrix(
    offsets,
    edges,
    weights
)
cost_matrix = waypoint_graph.compute_cost_matrix(target_locations)
target_map = {k:v for k, v in enumerate(target_locations)}

print(f"Index <-> Waypoint Mapping: \n{target_map}\n\n Waypoint Graph Cost Matrix: \n{cost_matrix}")

Index <-> Waypoint Mapping: 
{0: 0, 1: 4, 2: 5, 3: 6}

 Waypoint Graph Cost Matrix: 
     0    1    2    3
0  0.0  6.0  4.0  6.0
1  6.0  0.0  4.0  6.0
2  4.0  4.0  0.0  4.0
3  6.0  6.0  4.0  0.0


## Map Based

When dealing with problems in shipping and logistics, road distance and/or time is often used as a cost metric.  In these cases there are a number of tools available to calculate drive distance and/or time.  One such tool is the [Open Source Routing Machine](http://project-osrm.org/)(OSRM).  In the below example we create a cost matrix using OSRM from a list of lat/lon coordinates.

Ampol HQ -33.90717331392265, 151.1970757927905
Ampol Woolworths -33.90205317822677, 151.19997257845284
Ampol Woollahra -33.87636484074775, 151.2526643974862
Ampol Padstow -33.89278990156987, 151.0687065036508

#### Define Points of Interest

In [12]:
lat_lon_coords = [
    [-33.907173, 151.197075],
    [-33.9020531, 151.199972], 
    [-33.876364, 151.252664], 
    [-33.892789, 151.068706]
] 

#### Create Distance Matrix via OSRM

In [13]:
locations=""
for loc in lat_lon_coords:
    locations = locations + "{},{};".format(loc[1], loc[0])
r = requests.get("http://router.project-osrm.org/table/v1/driving/"+ locations[:-1])

routes = json.loads(r.content)
cols = [str(i) for i in lat_lon_coords]
cost_matrix = cudf.DataFrame(routes['durations'], columns = cols, index= cols)
print(f"Cost Matrix via OSRM:\n")
cost_matrix

Cost Matrix via OSRM:



,"[-33.907173, 151.197075]","[-33.9020531, 151.199972]","[-33.876364, 151.252664]","[-33.892789, 151.068706]"
"[-33.907173, 151.197075]",0.0,130.8,814.7,1438.2
"[-33.9020531, 151.199972]",131.4,0.0,769.4,1462.0
"[-33.876364, 151.252664]",812.9,752.3,0.0,1765.5
"[-33.892789, 151.068706]",1438.5,1439.1,1775.1,0.0


#### Map Visualization

Visualization can be a helpful tool for understanding and communication.  Here we demonstrate a sample visualization implementation showing the routes represented by the cost matrix above.

In [15]:
def get_map(my_lat_longs):
    m = folium.Map(location=[-33.9, 151.19], #[52.52, 13.41],
                   zoom_start=13)
    folium.Marker(
        location=[my_lat_longs[0][0],my_lat_longs[0][1]] ,
        icon=folium.Icon(icon='play', color='red')
    ).add_to(m)
    for loc in my_lat_longs[1:]:
        folium.Marker(
            location=[loc[0], loc[1]],
            icon=folium.Icon(icon='stop', color='green')
        ).add_to(m)
            
    for src_idx in range(len(lat_lon_coords)):
        for dst_idx in range(len(lat_lon_coords)):
            if src_idx == dst_idx:
                break
            source = lat_lon_coords[src_idx]
            destination = lat_lon_coords[dst_idx]
            loc = "{},{};{},{}".format(source[1], source[0], destination[1], destination[0])
            url = "http://router.project-osrm.org/route/v1/driving/"
            r = requests.get(url + loc) 

            res = r.json()   
            routes = polyline.decode(res['routes'][0]['geometry'])

            folium.PolyLine(
                routes,
                weight=5,
                color='blue',
                opacity=0.6
            ).add_to(m)

    return m
get_map(lat_lon_coords)